In [1]:
#Import required libraries
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns


In [2]:
#load the data
species = pd.read_csv('species_info.csv')
observations = pd.read_csv('observations.csv')


In [3]:
print(species.columns)
print(observations.columns)

Index(['category', 'scientific_name', 'common_names', 'conservation_status'], dtype='object')
Index(['scientific_name', 'park_name', 'observations'], dtype='object')


In [4]:
# check how many null values exist: 
# what are different data types in the data set?

print(observations.info(memory_usage= False))
print('\n')
print(species.info(memory_usage= False))

# 5824-191 = 5633: There are 5633 missing(null) rows in the conversation status.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23296 entries, 0 to 23295
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   scientific_name  23296 non-null  object
 1   park_name        23296 non-null  object
 2   observations     23296 non-null  int64 
dtypes: int64(1), object(2)None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5824 entries, 0 to 5823
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   category             5824 non-null   object
 1   scientific_name      5824 non-null   object
 2   common_names         5824 non-null   object
 3   conservation_status  191 non-null    object
dtypes: object(4)None


In [5]:
# Let's look at if missing data is systematic or if it's random before we try to deal with it.
print(species[species.conservation_status.notnull()])
print(species[species.conservation_status.isnull()])
species.conservation_status.value_counts(dropna=False)
#It appears that the missing rows are structurally missing data.

            category                   scientific_name  \
7             Mammal                     Canis latrans   
8             Mammal                       Canis lupus   
9             Mammal                       Canis rufus   
29            Mammal                  Eptesicus fuscus   
30            Mammal         Lasionycteris noctivagans   
...              ...                               ...   
5302  Vascular Plant             Callitriche marginata   
5399  Vascular Plant  Camissonia sierrae ssp. alticola   
5426  Vascular Plant                Epilobium oreganum   
5436  Vascular Plant              Botrychium ascendens   
5676  Vascular Plant                  Romneya coulteri   

                                           common_names conservation_status  
7                                                Coyote  Species of Concern  
8                                             Gray Wolf          Endangered  
9                                              Red Wolf          Enda

NaN                   5633
Species of Concern     161
Endangered              16
Threatened              10
In Recovery              4
Name: conservation_status, dtype: int64

In [27]:
# It appears that scientific name is common between two csv files/datasets. Let's count how many unique scientific names are in species dataframe
# and see if it matches total count of rows. If it does, it probably indicates that the species_info file is a database of all distinct species names. 

print(species.scientific_name.value_counts(dropna=False))
print(species.scientific_name.count()) # This is the total count of scientific_names 
species[species.scientific_name=='Castor canadensis'] # This is an example of a duplicate
# explore duplicated values 
species['dup_scientific_name']= species.scientific_name.duplicated(False)
species[species.dup_scientific_name == True]
#This showed that scientific name is duplicated when multiple common names exist for the same species
observations.scientific_name.nunique()
# There are 5541 unique species in the observations dataset


Castor canadensis            3
Canis lupus                  3
Hypochaeris radicata         3
Columba livia                3
Puma concolor                3
                            ..
Carex                        1
Hexastylis shuttleworthii    1
Hexastylis heterophylla      1
Hexastylis arifolia          1
Tribulus terrestris          1
Name: scientific_name, Length: 5541, dtype: int64
5824


5541

In [7]:
#Now let's look at the duplicates in the common_names column...

species['dup_common_name'] =species.common_names.duplicated(False)
print(species.common_names.value_counts(dropna=False)) 
# There are duplicates in the common_names as well. 
# Let's see the data to understand if they're typos.


Brachythecium Moss                                                            7
Dicranum Moss                                                                 7
Panic Grass                                                                   6
Bryum Moss                                                                    6
Sphagnum                                                                      6
                                                                             ..
Pine                                                                          1
Red Spruce                                                                    1
Norway Spruce                                                                 1
Fraser Fir                                                                    1
Bullhead, Caltrop, Goathead, Mexican Sandbur, Puncture Vine, Texas Sandbur    1
Name: common_names, Length: 5504, dtype: int64


In [8]:
duplicates_common_name = species[(species.dup_common_name == True) ]
duplicates_scientific_name = species[(species.dup_scientific_name == True) ]

In [9]:
# category, and conservation_status columns are categorical data so let's dig into them and see if there are any categorical names that has a typo
#  and if we can quickly clean that up. First let's count unique/distinct values


# observation seems to be a numerical value, so let's convert that to an integer, since it's a count.

In [25]:
#testing how to merge with duplicated scientific_names as that is the available common name between 2 datasets
test_species_df= duplicates_common_name[duplicates_common_name.scientific_name.str.contains('A[c-o]\w+', regex=True)]
test_species_df.sort_values(by='scientific_name',inplace=True)
test_species_df

/var/folders/1j/ldj8bssd0vs37fj53rh82w9w66dp51/T/ipykernel_58452/3996008913.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_species_df.sort_values(by='scientific_name',inplace=True)


,category,scientific_name,common_names,conservation_status,dup_scientific_name,dup_common_name
5677,Vascular Plant,Aconitum columbianum,Columbian Monkshood,NaN,False,True
4313,Vascular Plant,Aconitum columbianum ssp. columbianum,Columbian Monkshood,NaN,False,True
143,Bird,Actitis macularia,Spotted Sandpiper,NaN,False,True
4512,Bird,Actitis macularius,Spotted Sandpiper,NaN,False,True
730,Vascular Plant,Ageratina altissima,White Snakeroot,NaN,False,True
731,Vascular Plant,Ageratina altissima var. altissima,White Snakeroot,NaN,False,True
732,Vascular Plant,Ageratina altissima var. roanensis,White Snakeroot,NaN,False,True
3383,Vascular Plant,Agoseris glauca var. glauca,Pale Agoseris,NaN,False,True
4693,Vascular Plant,Agoseris glauca var. monticola,Pale Agoseris,NaN,False,True
3350,Vascular Plant,Allium cernuum,Nodding Onion,NaN,False,True
